# Steering Control for Udacity simulater using Deep learning

In [1]:
#import modules
import numpy as np
import keras
import csv
import cv2
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.models import *
from keras.layers import *
from keras.optimizers import Adam
import matplotlib.pyplot as plt

Using TensorFlow backend.


# Define the file directory for training data

In [20]:
features_directory = './train_data/'
labels_file= './train_data/driving_log.csv'

# Pre processing the image

We use 32*64 as the input layer dimension

In [3]:
rows = 32
cols = 64

Pre processing the image includes
1. cropping of the top and bottom of the frame since this is not useful for predictin the steering angle
2. resize and normalize the images
    Here only the S channel of the HSV image is used to reduce the parameters

In [4]:
#proprocess: change to HSV space and resize
def preprocess(img, top_offset=.375, bottom_offset=.125):
    """
    Applies preprocessing pipeline to an image: crops `top_offset` and `bottom_offset`
    portions of image, resizes to 32x128 px and scales pixel values to [0, 1].
    """
    top = int(top_offset * img.shape[0])
    bottom = int(bottom_offset * img.shape[0])
    resized = cv2.resize((cv2.cvtColor(img[top:-bottom, :], cv2.COLOR_RGB2HSV))[:,:,1],(cols,rows))
    return resized

# Function defintion to save model.json & model_h5

In [5]:
def model_save(model_json,model_h5):
    json_model = model.to_json()
    with open(model_json, "w") as f:
        f.write(json_model)
    model.save_weights(model_h5) 

# Loading data: steering tags and the  left, center & right camera 

The images from the left and right camera is also part of the training set. But only the center camera takes for prediction. For the input data we use the data provided by Udacity and add our training data by riding around the simulator circuit. 

Since the track didnt emulate shadows we randomly added shadows to the image vertically.
For each tag read from the input csv file the corresponding  image files are read. For the left and right cameras we give a steering offset of +/- delta to account for the difference.

In [13]:
def data_loading(delta):
    logs = []
    features = []
    labels = []
    augment = 1
    with open(labels_file,'rt') as f:
        temp = 0
        reader = csv.reader(f)
        
        for line in reader:
            logs.append(line)
        log_labels = logs.pop(0)
        shadow_indices = np.random.choice(2, len(logs), replace=True, p=[0.80, 0.20])
        for i in range(len(logs)):
        
            for j in range(3):
                img_path = logs[i][j]                    
                

                img_path = features_directory+'IMG'+(img_path.split('IMG')[1]).strip()
                img = plt.imread(img_path)    
                if augment:
                    # Add random shadow as a vertical slice of image
                    h, w = img.shape[0], img.shape[1]
                    [x1, x2] = np.random.choice(w, 2, replace=False)
                    k = h / (x2 - x1)
                    b = - k * x1
                    for row in range(h):
                        col = int((row - b) / k)
                        img.setflags(write=1)
                        img[row, :col, :] = (img[row, :col, :] * .5).astype(np.int32)
                features.append(preprocess(img))
                if j == 0:
                    labels.append(float(logs[i][3]))
                elif j == 1:
                    labels.append(float(logs[i][3]) + delta)
                else:
                    labels.append(float(logs[i][3]) - delta)
    return features, labels

In [14]:
#load the data and transform to numpy array
#parameter, defining the shift variable for left and righ steering angle
delta = 0.2
features, labels = data_loading(delta)

Convert features to array

In [15]:
features = np.array(features).astype('float32')
labels = np.array(labels).astype('float32')
print(features.shape)


(28437, 32, 64)


We can create double the training samples by just flipping the images horizontally

In [16]:
#augment the data by horizontal flipping the image
features = np.append(features,features[:,:,::-1],axis=0)
labels = np.append(labels,-labels,axis=0)
print(features.shape)

(56874, 32, 64)


Create the test and validation dataset

In [17]:
# shuffle the data and split to train and validation 
features, labels = shuffle(features, labels)
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, random_state=0, test_size=0.1)

#reshape the data  to feed into the network
train_features = train_features.reshape(train_features.shape[0], rows, cols, 1)
test_features = test_features.reshape(test_features.shape[0], rows, cols, 1)


# Building the Model 

Our model has 3 convolution layers and 3 fully connected layers.

In [18]:
#define the model
def steering_model():
    model = Sequential()
    model.add(Lambda(lambda x: x/127.5 - 1.,input_shape=(rows,cols,1)))	

    model.add(Convolution2D(8, 3, 3, init='normal',border_mode='valid'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2,2),border_mode='valid'))

    model.add(Convolution2D(8, 3, 3,init='normal',border_mode='valid'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2,2),border_mode='valid'))
    
    model.add(Convolution2D(8, 3, 3,init='normal',border_mode='valid'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2,2),border_mode='valid'))

    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(Activation('relu'))
    model.add(core.Dropout(.5))
    
    model.add(Dense(50))
    model.add(Activation('relu'))
    model.add(Dense(1))

    model.summary()
    return model

# Generating the model

We have used the standard Adam optimizer as in our labs with a leraning rate of 0.001.

For training and fitting the model we use a Batch size of 128 and number we utilized a couple of epochs but found that more than 15 there is no considerable difference in the performance. 

In [24]:
#optimize
model = steering_model()
adam = Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='mean_squared_error',optimizer='adam')
history = model.fit(train_features, train_labels,batch_size=128, epochs=11,verbose=1, validation_data=(test_features, test_labels))

#save the model architecture and parameters
model_json = './model.json'
model_h5 = './model.h5'
model_save(model_json,model_h5)
print ("Saved model to ", model_json)

c:\users\denny\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), kernel_initializer="normal", padding="valid")`
  
c:\users\denny\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D((2, 2), padding="valid")`
  
c:\users\denny\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), kernel_initializer="normal", padding="valid")`
  # Remove the CWD from sys.path while we load stuff.
c:\users\denny\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D((2, 2), padding="valid")`
  if sys.path[0] == '':
c:\users\denny\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `Conv2D`

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_4 (Lambda)            (None, 32, 64, 1)         0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 30, 62, 8)         80        
_________________________________________________________________
activation_16 (Activation)   (None, 30, 62, 8)         0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 15, 31, 8)         0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 13, 29, 8)         584       
_________________________________________________________________
activation_17 (Activation)   (None, 13, 29, 8)         0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 6, 14, 8)          0         
__________